<a href="https://colab.research.google.com/github/ellieyakim0829/dataAnalysis/blob/main/2_%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_OpenAPI_xml_%EC%8B%A4%EC%8A%B5_20240501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 분석

## 5. OpenAPI를 이용한 데이터 수집

### #그래프에서 한글사용하는 방법
- **(코랩에서)한글폰트 설치하기**

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 코랩에서 위 코드를 실행시킨 후  반드시 코랩 메뉴: "런타임>런타임 다시 시작" 합니다.

- **한글 폰트 지정하기**

In [1]:
# 코랩에서 한글 폰트 종류와 이름이 win과 다를 수 있다!!!
# 코랩: NanumGothic, 윈도우: Malgun Gothic
import matplotlib.pyplot as plt
plt.rcParams.update({'font.family': 'NanumBarunGothic',
                     'font.size': 12,
                     'figure.figsize': (6, 4),
                     'axes.unicode_minus':  False }) # 폰트 설정

---

### <a name="1)OpenAPI통한데이터수집">1) OpenAPI 통한 데이터 수집</a>

#### 1.도서관 정보나루  OpenAPI 신청하기
*  도서관 데이터 제공, 신청 활용 가능
    - 오픈 API 활용 방법: https://www.data4library.kr/apiUtilization
    - OpenAPI 신청하기: https://www.data4library.kr/myPage?isValid=true

#### 도서관 정보나루 API 사용하여 데이터 수집하기

In [ ]:
# requests 설치
!pip install requests

In [ ]:
import requests
import xml.etree.ElementTree as ET #Python의 표준 라이브러리 중 하나로 XML 데이터를 파싱하고, XML 트리를 생성하고, XML 문서를 생성하는 기능

# URL 설정
url = "http://data4library.kr/api/loanItemSrch?authKey=b8604a40cf9f0ce1e10c1d20ac9c70e6b6ddf0a5b7b751370b67131552680c20&startDt=2024-01-01&endDt=2024-04-30&gender=0;1&age=30;40;50&region=11"

# GET 요청을 보내고 응답 받기.
r = requests.get(url)
print(r)

# 응답의 인코딩을 UTF-8로 설정.
r.encoding = 'UTF-8'

# 응답을 XML 형식으로 파싱.
tree = ET.fromstring(r.text)
print(tree)

In [ ]:
import requests
import xml.etree.ElementTree as ET
from urllib.parse import urlparse, parse_qs

url = 'http://data4library.kr/api/loanItemSrch?authKey=b8604a40cf9f0ce1e10c1d20ac9c70e6b6ddf0a5b7b751370b67131552680c20&startDt=2024-01-01&endDt=2024-04-30&gender=0;1&age=30;40;50&region=11'

# URL을 파싱하여 쿼리 문자열의 매개변수를 가져오기.
parsed_url = urlparse(url)
query_params = parse_qs(parsed_url.query)

# 각 매개변수의 값을 가져오기
gender = query_params.get('gender', [''])[0]
age = query_params.get('age', [''])[0]
addCode = query_params.get('addCode', [''])[0]
kdc = query_params.get('kdc', [''])[0]
region = query_params.get('region', [''])[0]

print("Gender:", gender)
print("Age:", age)
print("Add Code:", addCode)
print("KDC:", kdc)
print("Region:", region)

# URL에서 XML 데이터를 추출.
response = requests.get(url)

# 응답의 텍스트를 XML 문자열로 변환.
xml_data = response.text

# XML 문자열을 파싱.
root = ET.fromstring(xml_data)

# 각 도서 정보에 접근하여 출력.
for doc in root.findall('docs/doc'):
    bookname = doc.find('bookname').text
    ranking = doc.find('ranking').text
    authors = doc.find('authors').text
    publisher = doc.find('publisher').text
    publication_year = doc.find('publication_year').text
    isbn13 = doc.find('isbn13').text
    loan_count = doc.find('loan_count').text
    print("도서명:", bookname)
    print("순위",ranking)
    print("저자:", authors)
    print("출판사:", publisher)
    print("출판년도:", publication_year)
    print("ISBN:", isbn13)
    print("대출 횟수:", loan_count)
    print("-----")

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

url = 'http://data4library.kr/api/loanItemSrch?authKey=b8604a40cf9f0ce1e10c1d20ac9c70e6b6ddf0a5b7b751370b67131552680c20&startDt=2024-01-01&endDt=2024-04-30&gender=0;1&age=30;40;50&region=11'

# URL에서 XML 데이터 추출
response = requests.get(url)

# XML 데이터를 파싱하여 ElementTree 객체로 변환
root = ET.fromstring(response.content)

# 'request' 요소에서 매개변수 정보를 추출하여 딕셔너리에 저장
params_data = {
    '성별': root.find('.//request/gender').text,
    '연령': root.find('.//request/age').text,
    '지역': root.find('.//request/region').text
    }

# 매개변수 정보를 DataFrame으로 변환
params_df = pd.DataFrame([params_data])

# 도서 정보를 저장할 리스트 초기화
books = []

# 각 도서 정보에 접근하여 리스트에 추가
for doc in root.findall('.//docs/doc'):
    bookname = doc.find('bookname').text
    ranking = doc.find('ranking').text
    authors = doc.find('authors').text
    publisher = doc.find('publisher').text
    publication_year = doc.find('publication_year').text
    isbn13 = doc.find('isbn13').text
    class_nm = doc.find('class_nm').text
    loan_count = doc.find('loan_count').text
    bookImageURL = doc.find('bookImageURL').text
    bookDtlUrl = doc.find('bookDtlUrl').text

    books.append({
        '도서명': bookname,
        '순위': ranking,
        '저자': authors,
        '출판사': publisher,
        '출판년도': publication_year,
        'ISBN': isbn13,
        '주제분류명':class_nm,
        '대출 횟수': loan_count,
        '책표지': bookImageURL,
        '도서상세페이지 URL': bookDtlUrl
    })

# 도서 정보를 DataFrame으로 변환
books_df = pd.DataFrame(books)

# 매개변수 정보와 도서 정보를 가로 방향으로 결합하여 하나의 DataFrame으로 만듭니다.
result_df = pd.concat([params_df, books_df], axis=1)
result_df.head(5)
# # 결과 DataFrame 출력
# print(result_df)

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

url = 'http://data4library.kr/api/loanItemSrch?authKey=b8604a40cf9f0ce1e10c1d20ac9c70e6b6ddf0a5b7b751370b67131552680c20&startDt=2023-04-01&endDt=2024-04-29&gender=0;1&age=8;20;30;40;50;60&region=11&addCode=0;1;2;4;5;6;7;9&kdc=0;1;2;3;4;5;6;7;8;9'

# URL에서 XML 데이터 추출
response = requests.get(url)

# XML 데이터를 파싱하여 ElementTree 객체로 변환
root = ET.fromstring(response.content)

# 도서 정보를 저장할 리스트 초기화
books = []

# 각 도서 정보에 접근하여 리스트에 추가
for doc in root.findall('.//docs/doc'):
    ranking = doc.find('ranking').text
    bookname = doc.find('bookname').text
    ranking = doc.find('ranking').text
    authors = doc.find('authors').text
    publisher = doc.find('publisher').text
    publication_year = doc.find('publication_year').text
    isbn13 = doc.find('isbn13').text
    class_nm = doc.find('class_nm').text
    loan_count = doc.find('loan_count').text
    bookImageURL = doc.find('bookImageURL').text
    bookDtlUrl = doc.find('bookDtlUrl').text

    books.append({
        '순위': ranking,
        '도서명': bookname,
        '순위': ranking,
        '저자': authors,
        '출판사': publisher,
        '출판년도': publication_year,
        'ISBN': isbn13,
        '주제분류명':class_nm,
        '대출 횟수': loan_count,
        '책표지': bookImageURL,
        '도서상세페이지 URL': bookDtlUrl
    })

# 도서 정보를 DataFrame으로 변환
books_df = pd.DataFrame(books)
books_df.head(5)
# DataFrame 출력
# print(books_df)

In [ ]:
result_df.info()

In [ ]:
!pip install missingno

In [ ]:
import missingno as msno
msno.matrix(result_df)
plt.show()

# 현재 DataFrame의 열 이름 확인

In [ ]:
print(f"현재 DataFrame의 열 이름:, {result_df.columns}")
result_df.head(3)

# 도서명에서 중복되는 값 제거 하기

In [ ]:
# 중복 되지 않은 값의 개수와  책 이름을 확인
# print( f'#고유한 값 갯수:{len(books_df.도서명.unique())}' )
# print( f'#고유한 책 이름명:{books_df.도서명.unique()}' )

# # '도서명' 열에서 중복되지 않는 고유한 값들을 추출하여 unique_books 변수에 저장
# unique_books = books_df['도서명'].unique()
# print(len(unique_books))

# # '도서명' 열을 기준으로 중복된 값을 제거하고 고유한 값들을 추출하여 unique_books_df 변수에 저장 새로운 DataFrame을 생성
unique_books_df = books_df.drop_duplicates(subset=['도서명'])
print(len(unique_books_df))
unique_books_df.head(10)




#### 6.가공된 파일 신규 저장하기

In [45]:
# 가공된 데이터 신규 파일로 저장하기
outfile = 'BestLoanList_books_data.csv'
unique_books_df.to_csv(outfile, encoding='cp949', mode='w', index=False)

#### 7.가공된 csv 파일 확인하기

In [ ]:
unique_books_df = pd.read_csv('BestLoanList_books_data.csv', encoding='cp949')

print(f"현재 DataFrame의 열 이름:, {unique_books_df.columns}")
unique_books_df.head(10)

# 막대 그래프로 표시하기

In [ ]:
import matplotlib.pyplot as plt

# 인기도서 순위 데이터 가져오기
rankings = unique_books_df['순위'].astype(int)
books_names = unique_books_df['도서명']

# 막대 그래프 그리기
plt.figure(figsize=(10, 6))
plt.barh(books_names.head(10), rankings.head(10), color='skyblue')  # head(10)을 사용하여 상위 10개만 선택
plt.xlabel('순위')
plt.ylabel('도서명')
plt.title('30대 40대 50대 서울시민 인기도서 순위 (상위 10개)')
plt.gca().invert_yaxis()  # y 축 역순으로 변경하여 1위가 위로 오도록 설정
plt.show()

# Pie 그래프로 표시하기

In [ ]:
# 순위별 도서 수 세기
# 상위 10개의 도서명과 해당 도서의 비율(퍼센트) 계산
top_10_books = unique_books_df['도서명'].head(10)
book_percentages = (unique_books_df['순위'].value_counts(normalize=True) * 100).head(10)

# 파이 그래프로 나타내기
plt.figure(figsize=(8, 8))
plt.pie(book_percentages, labels=top_10_books, autopct='%.3f%%', startangle=90, explode=[0.1]+[0]*9)
plt.title('30대 40대 50대 서울시민 인기도서 순위 (상위 10개) 비율')
plt.show()


##대출건수를 계산할 때 각 책을 구분할 수 있는 고유한 식별자가 주어짐
## - 가장 일반적인 고유한 식별자는 ISBN (International Standard Book Number)  활용하여 대출건수를 계산하면 된다.

##### * 1.먼저, 데이터를 읽어올 때 ISBN 컬럼을 사용하여 DataFrame을 생성하기
##### * 2.도서명 대신 ISBN을 사용하여 각 책의 대출건수를 계산.
##### * 3.필요에 따라 ISBN을 인덱스로 설정하여 데이터를 재구성.
##### * 4.이후에 대출건수 순위를 계산하고 시각화하기.

In [ ]:
# import pandas as pd

# # ISBN과 대출 횟수를 포함한 데이터 파일을 불러옵니다.
# books_data = pd.read_csv('BestLoanList_books_data.csv', encoding='cp949')
# print(books_data)

# # ISBN을 기준으로 대출 횟수와 도서명을 포함한 데이터를 생성.
books_with_loan_count = unique_books_df[['ISBN','대출 횟수', '도서명']]

# ISBN을 인덱스로 설정.
books_with_loan_count.set_index('ISBN', inplace=True)

# 대출 횟수에 따라 정렬한 뒤 상위 10개의 도서 생성.
top_10_books_with_loan_count = books_with_loan_count.sort_values(by='대출 횟수', ascending=False).head(10)

# 결과를 확인.
print(top_10_books_with_loan_count)
top_10_books_with_loan_count

In [ ]:
import matplotlib.pyplot as plt

# 상위 10개의 도서명과 대출 횟수를 추출합니다.
books_names = top_10_books_with_loan_count['도서명']
loan_counts = top_10_books_with_loan_count['대출 횟수']

# 막대 그래프 그리기
plt.figure(figsize=(12, 8))  # 그래프 크기 설정
plt.barh(books_names, loan_counts, color='skyblue')  # 가로 막대 그래프 그리기
plt.xlabel('대출 횟수')
plt.ylabel('도서명')
plt.title('서울시민 30대 40대 50대 인기도서 순위 (상위 10개)')
plt.gca().invert_yaxis()  # y 축 역순으로 변경하여 1위가 위로 오도록 설정

# 각 막대에 대출 횟수 표시
for i, v in enumerate(loan_counts):
    plt.text(v, i, str(v), color='black', va='center')

# 그래프 테마 설정
plt.grid(axis='x', linestyle='--', alpha=0.7)

# 그래프를 파일로 저장
plt.savefig('서울시민_30대 40대 50대_인기도서_10개_막대그래프.png')

# 그래프 보여주기
plt.show()

In [ ]:
# import matplotlib.pyplot as plt

# # 상위 10개의 도서명과 해당 도서의 비율(퍼센트) 계산
# top_10_books = top_10_books_with_loan_count['도서명'].head(10)
# book_percentages = (top_10_books_with_loan_count['대출 횟수'].value_counts(normalize=True) * 100).head(10)
# print(top_10_books)
# print(book_percentages)
# # 파이 그래프로 나타내기
# # plt.figure(figsize=(8, 8))
# # plt.pie(book_percentages, labels=top_10_books, autopct='%.3f%%', startangle=90, explode=[0.1]+[0]*9)
# # plt.title('30대 40대 50대 서울시민 인기도서 순위 (상위 10개) 비율')
# # plt.show()

import matplotlib.pyplot as plt

# 상위 10개의 도서명과 해당 도서의 비율(퍼센트) 계산
top_10_books = top_10_books_with_loan_count['도서명'].head(10)
book_percentages = (top_10_books_with_loan_count['대출 횟수'] / top_10_books_with_loan_count['대출 횟수'].sum() * 100).head(10)

# 파이 그래프로 나타내기
plt.figure(figsize=(8, 8))
plt.pie(book_percentages, labels=top_10_books, autopct='%.3f%%', startangle=90, explode=[0.1]+[0]*9)
plt.title('서울시민 30대 40대 50대 인기도서 순위 (상위 10개) 비율')

# 파일로 저장
plt.savefig('서울시민_30대 40대 50대_인기도서_10개_비율_파이그래프.png')

plt.show()

In [ ]:
# 라이브러리 설치하기
!pip install wordcloud

# 워드 클라우드 만들기

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 상위 10개의 도서명 추출
top_10_books_names = top_10_books_with_loan_count['도서명'].head(10)

# 워드 클라우드 생성
wordcloud = WordCloud(font_path='NanumBarunGothic.ttf', background_color='white').generate(' '.join(top_10_books_names))

# 워드 클라우드 표시
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('서울시민 30대 40대 50대 인기도서 순위 (상위 10개) ')
plt.axis('off')
plt.show()

# 파일로 저장하기
wordcloud.to_file('서울시민_30대 40대 50대_인기도서_10개_워드클라우드.png')

-------------------------